<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_basic_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

TensorFlow 2.x selected.
2.2.0-rc1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 0.25.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [0]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # 常量，这个函数就定义好了一个常量，用一个二维矩阵去初始化它

# index 这是我们在常量上做的索引操作
print(t) # 然后我们可以打印它
print(t[:, 1:]) #也可以有很多操作，向numpy那样，比如说我要取出第二列以后的数值
print(t[..., 1]) #比如说我要把第二列单独取出来，作为一个单独的tensor

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [0]:
# ops 在常量上做的算子操作,基本上tensorflow的所有的算子在这个constant上都会支持，
# 比如说加法，比如说平方，比如说矩阵乘以自己的转置tf.transpose呢就是做自己的转置
print(t+10)
print(tf.square(t))
print(t @ tf.transpose(t)) # 矩阵乘以自己的转置tf.transpose呢就是做自己的转置
# 其实呢，我们可以发现，已经用到了tensorflow2.0的新feature，也就是eager execution 
# 在1.0里面呢，如果我定义了一个常量，我调用他的print函数，是无法获得他具体的值的。因为我需要用session才能获取他具体的值
# 而在2.0里面呢，我可以定义完了以后呢，print完了以后呢，就可以获得他的值了。因为他的eager execution是默认打开的。
# 对应的，上面的op操作在1.0里面是需要在session里面去run一下，才能获取对应的值
# 而在tensorflow2.0里面呢，这些值是可以直接获得到的，就是你做完操作以后，直接可以获取到

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [0]:
# numpy conversion
# 在tensorflow2.0里面呢，还添加了tensorflow和numpy的转化操作
print(t.numpy()) # 比如说我在tensorflow里定义的一个常量，我可以直接用t.numpy方法获取到他的值
print(np.square(t)) # 同样的，对于一个tensor，我也可以把它作为一些numpy方法的输入。比如说一个tensor可以作为np.square的输入
np_t = np.array([[1., 2., 3.], [4., 5., 6.]]) # 我可以把一个numpy的对象呢，直接转成tensor
print(tf.constant(np_t)) # 我可以把一个numpy的对象呢，直接转成tensor，我们只需要在tf.constant里面传入这个numpy的数组就可以啦
# 如果在一个tensorflow tensor上面直接调用numpy，会直接获取他的值
# 如果是把一个tensor输入给numpy的一个函数呢，我们就会获取他算出来之后的那个值
# 如果我们把numpy传给tf.constant，我们就会获取一个tensor对象



[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [0]:
# Scalars 
# tensor可以是零维的，一维的，二维的，三维的，以至于n维的。 如果是零维的，怎么样个使用方法，
# 零维其实就是对应一个数，然后在tensorflow中成为scalar.如果我要建立一个0维的tensor的话，直接用tf.constant(数字)，
# 他的输入就是一个具体的数字就好了
t = tf.constant(2.718)
print(t.numpy())
print(t.shape) # 0维

2.718
()


In [0]:
# strings
# 在tensorflow中一个tensor不仅可以存储数字，还可以存储字符串
# 接下来看下如何和字符串做处理的。
t = tf.constant("cafe") # 输入是一个字符串
print(t)
print(tf.strings.length(t)) # 可以通过tf.string的相应函数，对这个字符串进行操作，
print(tf.strings.length(t, unit="UTF8_CHAR")) # 获取他utf编码时候的长度
print(tf.strings.unicode_decode(t, "UTF8")) # 我们使用unicode_decode来把我们字符串呢，从unicode呢转化为utf-8

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [0]:
# string array
t = tf.constant(["cafe", "coffee", "咖啡"]) # 除了一个字符串外，还可以存储一个数组的字符串
print(tf.strings.length(t, unit="UTF8_CHAR")) # 看下数组中每一个字符串的长度，这样就三个长度，
r = tf.strings.unicode_decode(t, "UTF8") # 将unicode 转成utf8
print(r) # 什么是RaggedTensor呢，就是不完整对象。因为每行的列数不完全一样，第二维长度不一样，这是tensorflow2.0里新加入的，是不规则的数据

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [2]:
# ragged tensor
r = tf.ragged.constant([[11, 12], [21, 22, 23], [], [41]]) #我们使用tf.ragged来初始化他，使用不规则数据
# index op
print(r)  # ragged tensor
print(r[1]) # 只取一行是个tensor
print(r[1:2]) # 第一行到第二行，左闭右开区间，是个ragged tensor

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>


In [3]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51, 52], [], [71]]) 
print(tf.concat([r, r2], axis = 0)) # ragged tensor支持拼接操作， r1和r2进行拼接，他们是在第0维度上进行拼接，效果就是按照行的方式拼接起来，拼接完就是7行

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>


In [4]:
r3 = tf.ragged.constant([[13, 14], [15], [], [42, 43]])
print(tf.concat([r, r3], axis = 1)) # 在第一维度上进行拼接，需要第一个维度呢，两个在第一个维度上呢 是一样的。即有一样的行数

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [], [41, 42, 43]]>


In [6]:
print(r)
print(r.to_tensor()) #regged tensor呢是可以转化为tensor的，只需要调to_tensor()函数就可以啦，缺少的数据用0补齐
# 因为用0补齐，所以0都在正常值的后面

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


In [7]:
# sparse tensor
s = tf.SparseTensor(indices = [[0, 1], [1, 0], [2, 3]], #第一个参数： 三个index, # 在一个非常大的二维矩阵里，只有几个值有数字，其他都为0，那么如何存储矩阵呢。
                    values = [1., 2., 3.],#第二个参数： 这三个坐标位置的值 # 这种数据不适合用raggedTensor，因为raggedTensor填充值都在后面，
                    dense_shape = [3, 4]) #第三个参数： 这个矩阵的具体的大小，比如是一个3*4的矩阵 # 可以用sparse tensor.在一个非常大的二维矩阵里，只有几个值有数字，其他都为0
                                  # 那么我就把这些位置的坐标加上他的值 存储起来
print(s)
print(tf.sparse.to_dense(s)) #将sparetensor 转成普通的 tensor

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [0]:
# ops on sparse tensors

s2 = s * 2.0 #sparse tensor 上的算子类型，可以做乘法
print(s2)

try:
    s3 = s + 1  # 不支持加法， 加法有问题
except TypeError as ex:
    print(ex)

s4 = tf.constant([[10., 20.],
                  [30., 40.],
                  [50., 60.],
                  [70., 80.]]) #定义一个4*几的矩阵 这里是4*2的矩阵
print(tf.sparse.sparse_dense_matmul(s, s4)) #s是一个3*4的矩阵，s跟s4相乘，得到的是一个普通3*2的 tensor

#还有一个问题，在定义sparse tensor的时候 index必须是排好序的，如果没有排好序的话，虽然不影响sparse tensor的使用，但是如果调用toTense函数的话，他就会发生错误

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [10]:
# sparse tensor
s5 = tf.SparseTensor(indices = [[0, 2], [0, 1], [2, 3]], # 这里不让tensor Index排好序
                    values = [1., 2., 3.],
                    dense_shape = [3, 4])
print(s5) # s5可以被打印出来
#print(tf.sparse.to_dense(s5)) # 但是sparse_to_tenser的时候就会出错
s6 = tf.sparse.reorder(s5)  # 我们需要对s5 reorder 就可以了
print(tf.sparse.to_dense(s6)) # 在用s6调用spare_to_tense就没问题了 这是spase_to_tenser的一个比较常见的坑

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [11]:
# Variables
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]]) # 上面将的都是常量。在神经网络中常量是不会变得。但是实际我们得参数是要更新得，这时候就需要Variable
print(v) # 一正常打印，他会获得一个tf.Variable对像
print(v.value()) #把Variables变成tensor,这就跟普通的常量是一样的
print(v.numpy()) #把Variables变成numpy，把具体的值打印出来

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [12]:
# assign value  # 变量比常量多了一些操作，比如说重新赋值
v.assign(2*v)    # 通过assign函数重新赋值
print(v.numpy()) # 打印具体值
v[0, 1].assign(42) #变量的某一个位置重新赋值
print(v.numpy()) # 打印具体值
v[1].assign([7., 8., 9.]) #变量的某一行重新赋值
print(v.numpy()) # 打印具体值

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 7.  8.  9.]]


In [13]:
try:
    v[1] = [7., 8., 9.] # tf.variable变量的赋值只能用assign函数，用等于号是不可以的
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
